In [1]:
import os
os.environ["HF_DATASETS_OFFLINE"] = "1"
import datasets
import json

/Users/admin/fish/planck-agent/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

zh_hard = datasets.load_dataset("bz-arc13/zp_fc_gpt4")["train"]

Using the latest cached version of the dataset since bz-arc13/zp_fc_gpt4 couldn't be found on the Hugging Face Hub (offline mode is enabled).
Found the latest cached dataset configuration 'default' at /Users/admin/.cache/huggingface/datasets/bz-arc13___zp_fc_gpt4/default/0.0.0/e50892c581048be7bca0d685ac48a98b151f1732 (last modified on Thu Dec  5 15:36:04 2024).


In [3]:
def convert(data):
    msgs = data["conversation"]
    new_msgs = []
    for d in msgs:
        if fc := d["function_call"]:
            fc["arguments"] = json.loads(fc["arguments"])
            d["function_call"] = json.dumps([fc], ensure_ascii=False)
        new_msgs.append(d)
    data["conversation"] = new_msgs
    return data

new_ds = zh_hard.map(convert, num_proc=10)
new_ds

Dataset({
    features: ['functions', 'conversation'],
    num_rows: 3064
})

In [4]:
new_ds[0]

{'functions': '[{"name": "analyze_user_demographics", "description": "分析用户人口统计数据以识别目标市场", "parameters": {"type": "object", "properties": {"age_ranges": {"type": "array", "description": "年龄范围列表", "items": {"type": "object", "properties": {"min_age": {"type": "integer", "description": "最小年龄"}, "max_age": {"type": "integer", "description": "最大年龄"}}, "required": ["min_age", "max_age"]}}, "gender": {"type": "array", "description": "性别列表", "items": {"type": "string", "enum": ["male", "female", "other"]}}, "interests": {"type": "array", "description": "兴趣列表", "items": {"type": "string", "description": "用户兴趣"}}, "location": {"type": "object", "description": "地理位置信息", "properties": {"country": {"type": "string", "description": "国家"}, "city": {"type": "string", "description": "城市"}}, "required": ["country"]}}, "required": ["age_ranges", "gender", "interests", "location"]}}, {"name": "identify_growth_channels", "description": "识别潜在的用户增长渠道", "parameters": {"type": "object", "properties": {"existin

In [5]:
new_ds.push_to_hub("hqfx/fc_zh_hard")

Uploading the dataset shards: 100%|██████████| 1/1 [00:09<00:00,  9.85s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/hqfx/fc_zh_hard/commit/7b035af68dafd9cf82126793354c0ab0d12289b1', commit_message='Upload dataset', commit_description='', oid='7b035af68dafd9cf82126793354c0ab0d12289b1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/hqfx/fc_zh_hard', endpoint='https://huggingface.co', repo_type='dataset', repo_id='hqfx/fc_zh_hard'), pr_revision=None, pr_num=None)